# Python-Tipps

Eine detaillierte Einführung in Python kann hier nicht gegeben werden. In Ergänzung zu den üblichen Konstrukten wie Bedingungen (`if`), Schleifen (`for`) und Datenstrukturen wie Listen und Dictionaries gibt es aber ein paar Kniffe, die in diesem Kontext vieles erleichtern. Sie sind an dieser Stelle für ein besseres Verständnis noch einmal zusammengetragen.

## List Comprehension

Daten müssen für die Analyse häufig aufbereitet werden. Zwei typische Aufbereitungsprozeduren sind Filtern und Abbilden. Beim Filtern werden nur Elemente beibehalten, die ein bestimmtes Kriterium erfüllen, beim Abbilden wird jedes Element einer Liste über eine Funktion in ein neues Element transformiert.

Python stellt eine besondere Syntax bereit, die für beide Zwecke verwendet werden kann.

In [1]:
l = ['a', 'bbb', 'c', 'ddd']

In [2]:
# Transformation
[s.upper() for s in l]

['A', 'BBB', 'C', 'DDD']

In [3]:
# Filtern
[s for s in l if len(s) > 1]

['bbb', 'ddd']

In [4]:
# Beides auf einmal
[s.upper() for s in l if len(s) > 1]

['BBB', 'DDD']

In [5]:
# Und sogar für dictionaries
{s: s.upper() for s in l}

{'a': 'A', 'bbb': 'BBB', 'c': 'C', 'ddd': 'DDD'}

## String-Formatierung

Oftmals sollen Daten auf bestimmte Weise für die Ausgabe formatiert werden. Dazu ist eine übliche Technik, eine Art Vorlage-Zeichenkette mit Platzhaltern zu erstellen, für die dann bestimmte Werte eingesetzt werden.

Dazu gibt es in Python (mindestens) zwei Möglichkeiten. Älter, aber immer noch häufig gelehrt, ist diese Form:

In [6]:
u'%s kosten %.2f Mark.' % ('Bananen', 3.2)

u'Bananen kosten 3.20 Mark.'

Neuer ist diese Variante, die hier durchgehend verwendet wird:

In [7]:
u'{} kosten {:.2f} Euro.'.format('Mangos', 4.9)

u'Mangos kosten 4.90 Euro.'

## Generatoren

### Listen und Generatoren

Häufig sind Daten als Reihe von Werten strukturiert. Eine typische Form einer solchen Reihe ist eine Liste. Listen erlauben zwei Zugriffsformen: Das Iterieren (alle Werte nacheinander aufrufen) mit `for x in y` und den direkten Zugriff auf ein bestimmtes Element mit `y[i]`.

In [8]:
r = range(3)
r

[0, 1, 2]

In [9]:
for i in r:
    print i

0
1
2


In [10]:
r[1]

1

Dafür muss aber die ganze Liste vorgehalten werden. Bei sehr großen Datenstrukturen oder bei rechenintensiven Prozeduren kann es sinnvoll sein, die Elemente erst nacheinander zu erzeugen. Das setzt etwa networkx um bei der Berechnung von Teilgraphen zusammenhängender Netzwerk-Komponenten, oder gensim in seinem TextCorpus-Format. Diese »Generatoren« erlauben zwar das Iterieren, nicht aber den direkten Elementzugriff (weil gar nicht bekannt ist, welches Element an einer bestimmten Stelle sitzt).

Generatoren sind einfache Funktionen und man kann sie leicht selbst erstellen. Dazu erstellt man nicht eine Liste, die man dann mit `return` zurückgibt, sondern man gibt die Elemente nacheinander mit `yield` aus.

In [11]:
def grange(x):
    for i in range(x):
        yield i

r = grange(3)
r

<generator object grange at 0x7fce440c2780>

In [12]:
for i in r:
    print i

0
1
2


In [13]:
r[1]

TypeError: 'generator' object has no attribute '__getitem__'

Manchmal ist nur das erste Element eines Generators relevant, etwa wenn man das Teil-Netzwerk für die größte Komponente in einem Netzwerk erhalten will, aber nicht an den darauf folgenden, kleineren Komponenten interessiert ist. Hier hift ein Kniff, bei dem man eine Schleife direkt nach dem ersten Durchgang abbricht:

In [14]:
r = grange(3)
for i in r:
    break
i

0

## Lambda-Ausdrücke

Gelegentlich muss man einer Funktion eine Funktion übergeben. Die Funktion `sorted()` hat z.B. einen optionalen Parameter `key`. Hier kann man eine Funktion übergeben, die auf jedes Element angewandt wird, um die Sortierreihenfolge zu bestimmen. Normaler Weise werden etwa Großbuchstaben vor Kleinbuchstaben einsortiert:

In [15]:
l = ['ab', 'Ac']
sorted(l)

['Ac', 'ab']

Gelegentlich will man aber beim Sortieren die Groß- und Kleinschreibung ignorieren. Dann kann man die Funktion `str.lower` übergeben, damit die Werte für die Sortierung als Kleinbuchstaben behandelt werden.

In [16]:
sorted(l, key=str.lower)

['ab', 'Ac']

Komplexere Fälle erfordern komplexere Lösungen. So werden Tupel standardmäßig nach ihrem ersten Element sortiert:

In [17]:
l = [('c', 2), ('a', 3), ('b', 1)]
sorted(l)

[('a', 3), ('b', 1), ('c', 2)]

Um nach dem zweiten zu sortieren, wäre eine Funktion erforderlich, die für jedes Tupel das zweite Element zurückgibt.

In [18]:
def sort_by_second(x):
    return x[1]

sorted(l, key=sort_by_second)

[('b', 1), ('c', 2), ('a', 3)]

Es ist aber sehr umständlich, in diesen Fällen jedes Mal eine Funktion zu definieren, die man ggf. nur einmal benötigt. Für sehr einfache Funktionen, die nur aus einem einzelnen Ausdruck bestehen, gibt es daher in Python die Möglichkeit, mit `lambda` adhoc eine lokale Funktion zu definieren.

In [19]:
sorted(l, key=lambda x: x[1])

[('b', 1), ('c', 2), ('a', 3)]

## Formatierung in IPython Notebooks

Manchmal sollen Werte nicht nur als reiner Text ausgegeben werden, sondern mit ein wenig Formatierung. In IPython Notebooks kann man dies über HTML erreichen:

In [20]:
from IPython.core.display import HTML

result = '<ul>'
for i in range(3):
    result += '<li>{}</li>'.format(i)
result += '</ul>'

HTML(result)

Leider ist HTML relativ umständlich. Eine Alternative dazu ist Markdown. Diese simple Formatierungssprache lässt sich leichter erzeugen. IPython Notebooks nutzt das Format z.B. in seinen Textfeldern. Es gibt zwar keine vordefinierte Funktion für die Ausgabe von Markdown, aber eine solche lässt sich sehr leicht erstellen:

In [21]:
from IPython.nbconvert.filters import markdown2html

class MD(unicode):
    def _repr_html_(self):
        return markdown2html(self)

result = ''
for i in range(3):
    result += '* {}\n'.format(i)

MD(result)

/home/frederik/Dropbox/Dokumente/Lehrveranstaltungen/LA_Luzern/Python/pyenv/local/lib/python2.7/site-packages/IPython/nbconvert/filters/markdown.py:78: UserWarning: Node.js 0.9.12 or later wasn't found.
Nbconvert will try to use Pandoc instead.
  "Nbconvert will try to use Pandoc instead.")


u'* 0\n* 1\n* 2\n'